In [1]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import numpy as np
import pandas.io.json as pd_json
from sqlalchemy import create_engine


In [2]:
jsonfile = pd.read_json("Resources/mergedjson.json")
jsonfile

modifiedjson = jsonfile.drop(['target_fg','target_og','contributed_by','boil_volume', 'method', 'volume'], axis = 1)

In [3]:
# NEED TO FILL THE NaNS HERE

In [4]:
#function to scale srm values based on scale
#https://www.twobeerdudes.com/beer/srm
def srmCat(s):
    if s['srm'] >= 1 and s['srm'] < 3:
        return 'Pale Lager, Pilsner, Witbier, Berliner Weisse'
    elif 3 >= s['srm'] < 4:
        return 'Straw'
    elif 4 >= s['srm'] < 6:
        return 'Pale Gold'
    elif 6 >= s['srm'] < 8:
        return 'Deep Gold'
    elif 8 >= s['srm'] < 10:
        return 'Pale Amber'
    elif 10 >= s['srm'] < 13:
        return 'Medium Amber'
    elif 13 >= s['srm'] < 17:
        return 'Deep Amber'
    elif 17 >= s['srm'] < 20:
        return 'Amber Brown'
    elif 20 >= s['srm'] < 24:
        return 'Brown'
    elif 24 >= s['srm'] < 35:
        return 'Ruby Brown'
    elif 35 >= s['srm'] < 40:
        return 'Deep Brown'
    elif s['srm'] >= 40:
        return 'Black'
    else:
        return 'Undefined'
    
modifiedjson['srm_category'] = modifiedjson.apply(srmCat, axis=1)

In [5]:
## lk
#function to turn SRM categories into a number
def srmCat2(i):
    if i['srm_category'] == 'Pale Lager, Pilsner, Witbier, Berliner Weisse':
        return 1
#   elif 25 >= i['ibu'] < 40:
    elif i['srm_category'] == 'Straw':
        return 2
    elif i['srm_category'] == 'Pale Gold':
        return 3
    elif i['srm_category'] == 'Deep Gold':
        return 4
    elif i['srm_category'] == 'Pale Amber':
        return 5
    elif i['srm_category'] == 'Medium Amber':
        return 6
    elif i['srm_category'] == 'Deep Amber':
        return 7
    elif i['srm_category'] == 'Brown':
        return 8
    elif i['srm_category'] == 'Ruby Brown':
        return 9
    elif i['srm_category'] == 'Deep Brown':
        return 10
    elif i['srm_category'] == 'Black':
        return 11
    else:
        return 12
    
modifiedjson['srm_category_nbr'] = modifiedjson.apply(srmCat2, axis =1)

In [6]:
# getting rid of the NaNs in the IBU and ABV columns with column means
modifiedjson=modifiedjson.fillna(modifiedjson.mean())

In [7]:
#function to scale ibu into 3 categories 
def ibuCat(i):
    if i['ibu'] >= 0 and i['ibu'] < 25:
        return 'IBU Low'
#   elif 25 >= i['ibu'] < 40:
    elif i['ibu'] >= 25 and i['ibu'] < 40:
        return 'IBU Medium'
    elif i['ibu'] >=40:
        return 'IBU High'
    else:
        return 'IBU Undefined'
    
modifiedjson['ibu_category'] = modifiedjson.apply(ibuCat, axis =1)

In [8]:
## lk
#function to turn ibu categories into a number
def ibuCat2(i):
    if i['ibu_category'] == 'IBU Low':
        return 1
#   elif 25 >= i['ibu'] < 40:
    elif i['ibu_category'] == 'IBU Medium':
        return 2
    elif i['ibu_category'] == 'IBU High':
        return 3
    else:
        return 4
    
modifiedjson['ibu_category_nbr'] = modifiedjson.apply(ibuCat2, axis =1)

In [9]:
#function to scale abv into 3 categories 
def abvCat(a):
    if a['abv'] >= 0 and a['abv'] < 9:
        return 'ABV Low'
#   elif 9 >= a['abv'] < 16:
    elif a['abv'] >= 9 and a['abv'] < 16:
        return 'ABV Medium'
    elif a['abv'] >=16:
        return 'ABV High'
    else:
        return 'ABV Undefined'

modifiedjson['abv_category'] = modifiedjson.apply(abvCat, axis =1)   

In [10]:
## lk
#function to turn ibu categories into a number
def abvCat2(a):
    if a['abv_category'] == 'ABV Low':
        return 1
    elif a['abv_category'] == 'ABV Medium':
        return 2
    elif a['abv_category'] == 'ABV High':
        return 3
    else:
        return 4

modifiedjson['abv_category_nbr'] = modifiedjson.apply(abvCat2, axis =1)   

In [11]:
#70-82
def attenCat(a):
    if a['attenuation_level'] >=0.0 and a['attenuation_level'] < 70.0:
        return 'Atten Low'
    elif a['attenuation_level'] >= 70.0 and a['attenuation_level'] < 82.0:
        return 'Atten Medium'
    elif a['attenuation_level'] >= 82.0:
        return 'Atten High'
    else:
        'Undefined'
        
modifiedjson['attenuation_category'] = modifiedjson.apply(attenCat, axis =1)    

In [12]:
## lk
#function to turn attenuation categories into a number
def attenCat2(i):
    if i['attenuation_category'] == 'Atten Low':
        return 1
#   elif 25 >= i['ibu'] < 40:
    elif i['attenuation_category'] == 'Atten Medium':
        return 2
    elif i['attenuation_category'] == 'Atten High':
        return 3
    else:
        return 4
    
modifiedjson['atten_category_nbr'] = modifiedjson.apply(attenCat2, axis =1)

In [13]:
beerdata = modifiedjson[['id','name','ibu','ibu_category','ibu_category_nbr','srm','srm_category','srm_category_nbr','abv','abv_category','abv_category_nbr', 'ebc','attenuation_level','attenuation_category','atten_category_nbr','description',
                        'image_url','ph', 'tagline','brewers_tips']].set_index('id')
beerdata.to_csv('beerdata_lk.csv')
beerdata.dtypes

name                     object
ibu                     float64
ibu_category             object
ibu_category_nbr          int64
srm                     float64
srm_category             object
srm_category_nbr          int64
abv                     float64
abv_category             object
abv_category_nbr          int64
ebc                     float64
attenuation_level       float64
attenuation_category     object
atten_category_nbr        int64
description              object
image_url                object
ph                      float64
tagline                  object
brewers_tips             object
dtype: object

In [14]:
with open('Resources/mergedjson.json', encoding='utf8') as data_file:
    data = json.load(data_file)

maltdf = json_normalize(data, meta = ['id'], record_path=['ingredients',['malt']],errors='ignore')
maltdf.drop(columns=['amount'])

maltdf = maltdf.groupby(maltdf['id']).agg(','.join)
maltdf = maltdf.rename(columns={'name': 'malts_name'})
maltdf

hopsdf = json_normalize(data, meta = ['id'], record_path =['ingredients', ['hops']], errors = 'ignore')
hopsdf.drop(columns=['amount'])
hopsdf = hopsdf.groupby(hopsdf['id']).agg(','.join)
hopsdf = hopsdf.rename(columns={'add':'hops_add', 'attribute':'hops_attribute', 'name': 'hops_name'})

fooddf = json_normalize(data, meta =['id'], record_path=['food_pairing'], errors = 'ignore')
fooddf = fooddf.groupby(fooddf['id']).agg(','.join)
fooddf = fooddf.rename(columns = {0:'food_pairing'})

ingredf = maltdf.merge(hopsdf, how ='outer',left_index=True, right_index=True)
ingredf = ingredf.merge(fooddf, how = 'outer', left_index=True, right_index=True)
finaldata = beerdata.merge(ingredf, how= 'outer', left_index=True, right_index=True)

finaldata.to_csv('finalbeerdata.csv')


In [15]:
# connection_string = "db/beerdata.sqlite"
# engine = create_engine(f'sqlite:///{connection_string}')
# finaldata.to_sql(name='finaldata', con=engine, if_exists='append', index=True)

In [16]:
# working on decision tree (unit 21, 2-4) - i don't think RandomForest will work
from sklearn import tree
import os


In [37]:
# data2 = data2.drop("ibu_category", axis=1)
data2 = finaldata[['ibu','ibu_category_nbr', 'srm', 'srm_category_nbr', 'abv','abv_category_nbr', 'attenuation_level','atten_category_nbr']]
feature_names = data2.columns
data2.dtypes

ibu                   float64
ibu_category_nbr        int64
srm                   float64
srm_category_nbr        int64
abv                   float64
abv_category_nbr        int64
attenuation_level     float64
atten_category_nbr      int64
dtype: object

In [18]:
# data2 = data2.drop("ibu_category", axis=1)
# data2 = data2.drop("srm_category", axis=1)
# data2 = data2.drop("abv_category", axis=1)
# data2 = data2.drop("attenuation_level", axis=1)

In [38]:
data2.to_csv('data2.csv')
data2

,ibu,ibu_category_nbr,srm,srm_category_nbr,abv,abv_category_nbr,attenuation_level,atten_category_nbr
id,,,,,,,,
1,60.00000,3,10.000000,6,4.50,1,75.0,2
2,41.50000,3,15.000000,12,4.10,1,76.0,2
3,8.00000,1,4.000000,3,4.20,1,83.0,3
4,55.00000,3,15.000000,12,6.30,1,80.0,2
5,59.00000,3,5.000000,4,7.20,1,67.0,1
6,38.00000,2,7.500000,5,5.20,1,88.9,3
7,35.00000,2,40.000000,11,11.20,2,84.0,3
8,40.00000,3,6.000000,4,4.70,1,78.0,2
9,30.00000,2,42.000000,11,12.50,2,83.0,3


In [39]:
# data2.mean()

In [40]:
# data2=data2.fillna(data2.mean())
# data2.to_csv('data2.csv')
# data2

#### THIS COMMAND IS TOO LATE AND DOESN'T IMPACT THE CATEGORY NAME OR NBR DOWN HERE.

In [41]:
# data2.to_csv('data2.csv')
# data2

In [42]:
# examples from activity - unit 21/2-04
target = data2["ibu_category_nbr"]
target_names = [1,2,3,4]


In [43]:
test_data = data2.drop("ibu_category_nbr", axis=1)
feature_names = test_data.columns
test_data.head()

,ibu,srm,srm_category_nbr,abv,abv_category_nbr,attenuation_level,atten_category_nbr
id,,,,,,,
1,60.0,10.0,6,4.5,1,75.0,2
2,41.5,15.0,12,4.1,1,76.0,2
3,8.0,4.0,3,4.2,1,83.0,3
4,55.0,15.0,12,6.3,1,80.0,2
5,59.0,5.0,4,7.2,1,67.0,1


In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(test_data, target, random_state=42)

In [45]:
# accuracy of predicting IBU Category
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [46]:
# examples from activity - unit 21/2-04
target2 = data2["abv_category_nbr"]
target_names2 = [1,2,3,4]

In [47]:
test_data2 = data2.drop("abv_category_nbr", axis=1)
feature_names2 = test_data2.columns
test_data2.head()

,ibu,ibu_category_nbr,srm,srm_category_nbr,abv,attenuation_level,atten_category_nbr
id,,,,,,,
1,60.0,3,10.0,6,4.5,75.0,2
2,41.5,3,15.0,12,4.1,76.0,2
3,8.0,1,4.0,3,4.2,83.0,3
4,55.0,3,15.0,12,6.3,80.0,2
5,59.0,3,5.0,4,7.2,67.0,1


In [48]:
# from sklearn.model_selection import train_test_split (already brought in)
X_train, X_test, y_train, y_test = train_test_split(test_data2, target2, random_state=42)

In [49]:
# accuracy of predicting ABV Category
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [50]:
# examples from activity - unit 21/2-04
target3 = data2["atten_category_nbr"]
target_names3 = [1,2,3,4]

In [51]:
test_data3 = data2.drop("atten_category_nbr", axis=1)
feature_names3 = test_data3.columns
test_data3.head()

,ibu,ibu_category_nbr,srm,srm_category_nbr,abv,abv_category_nbr,attenuation_level
id,,,,,,,
1,60.0,3,10.0,6,4.5,1,75.0
2,41.5,3,15.0,12,4.1,1,76.0
3,8.0,1,4.0,3,4.2,1,83.0
4,55.0,3,15.0,12,6.3,1,80.0
5,59.0,3,5.0,4,7.2,1,67.0


In [52]:
# from sklearn.model_selection import train_test_split (already brought in)
X_train, X_test, y_train, y_test = train_test_split(test_data3, target3, random_state=42)

In [53]:
# accuracy of predicting Attenuation Category
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [54]:
# examples from activity - unit 21/2-04
target4 = data2["atten_category_nbr"]
target_names4 = [1,2,3,4]

In [55]:
test_data4 = data2.drop("srm_category_nbr", axis=1)
feature_names4 = test_data4.columns
test_data4.head()

,ibu,ibu_category_nbr,srm,abv,abv_category_nbr,attenuation_level,atten_category_nbr
id,,,,,,,
1,60.0,3,10.0,4.5,1,75.0,2
2,41.5,3,15.0,4.1,1,76.0,2
3,8.0,1,4.0,4.2,1,83.0,3
4,55.0,3,15.0,6.3,1,80.0,2
5,59.0,3,5.0,7.2,1,67.0,1


In [56]:
# from sklearn.model_selection import train_test_split (already brought in)
X_train, X_test, y_train, y_test = train_test_split(test_data3, target3, random_state=42)

In [57]:
# accuracy of predicting Attenuation Category
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0